In [ ]:
import os
import google.generativeai as genai
# os.environ["GOOGLE_API_KEY"] = "xxxx"


genai.configure(api_key=os.environ["GOOGLE_API_KEY"])





In [4]:
import os
import json
import time
import fitz  # PyMuPDF
import shutil
import tempfile
from urllib.parse import urljoin, urlparse, parse_qs
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc

from bs4 import BeautifulSoup

def table_html_to_numbered_list(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", id="MainContent_GridViewStatus")
    if not table:
        return []

    numbered_list = []
    rows = table.find_all("tr")[1:]  # skip header row

    for i, row in enumerate(rows, start=1):
        cells = row.find_all("td")
        if len(cells) >= 3:
            date = cells[0].get_text(strip=True)
            chamber = cells[1].get_text(strip=True)
            status = cells[2].get_text(strip=True)
            numbered_list.append(f"{i}. {date} | {chamber} | {status}")

    return numbered_list

from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_measure_links(html, base_url):
    soup = BeautifulSoup(html, "html.parser")
    links = []

    # 1️⃣ Links inside div.noprint
    noprint_div = soup.find("div", class_="noprint")
    if noprint_div:
        for a in noprint_div.find_all("a", href=True):
            links.append(urljoin(base_url, a["href"]))

    # 2️⃣ Links inside subsequent divs with class starting with 'measure-status card shadow'
    measure_divs = soup.find_all("div", class_="measure-status card shadow")
    # Include possible variation with "text-center"
    measure_divs += soup.find_all("div", class_="measure-status card shadow text-center")

    for div in measure_divs:
        for a in div.find_all("a", href=True):
            links.append(urljoin(base_url, a["href"]))

    # remove duplicates while preserving order
    seen = set()
    unique_links = []
    for link in links:
        if link not in seen:
            unique_links.append(link)
            seen.add(link)

    return unique_links

def extract_measure_documents_with_links(html, base_url):
    """
    Extract documents along with their URLs.
    Returns a list of dicts: [{"name": "HB400_HD1", "url": "..."}]
    """
    soup = BeautifulSoup(html, "html.parser")
    documents = []

    # 1️⃣ Documents inside div.noprint
    noprint_div = soup.find("div", class_="noprint")
    if noprint_div:
        for a in noprint_div.find_all("a", href=True):
            name = a.get_text(strip=True)
            url = urljoin(base_url, a["href"])
            if name:
                documents.append({"name": name, "url": url})

    # 2️⃣ Documents inside divs with class starting with 'measure-status card shadow'
    measure_divs = soup.find_all("div", class_="measure-status card shadow")
    # measure_divs += soup.find_all("div", class_="measure-status card shadow text-center")
    for div in measure_divs:
        for a in div.find_all("a", href=True):
            name = a.get_text(strip=True)
            url = urljoin(base_url, a["href"])
            if name:
                # Avoid duplicates
                if not any(d["name"] == name for d in documents):
                    documents.append({"name": name, "url": url})

    return documents


# Setup output filename
# measure_url = "https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=HB&billnumber=727&year=2025"
measure_url = "https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=HB&billnumber=400&year=2025"
parsed = urlparse(measure_url)
params = parse_qs(parsed.query)
billtype = params.get("billtype", ["UNKNOWN"])[0]
billnumber = params.get("billnumber", ["UNKNOWN"])[0]
year = params.get("year", ["UNKNOWN"])[0]
output_filename = f"{billtype}_{billnumber}_{year}.json"

# Setup download directory
download_dir = tempfile.mkdtemp()

options = uc.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "plugins.always_open_pdf_externally": True
})
driver = uc.Chrome(options=options)

def clean_html_text(html):
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()
    return soup.get_text(separator="\n", strip=True)

def extract_pdf_text_from_file(file_path):
    try:
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    except Exception as e:
        return f"[ERROR extracting PDF text: {e}]"

try:
    driver.get(measure_url)
    time.sleep(0.1)

    # Collect all links inside main-content
    main = driver.find_element(By.ID, "main-content")
    a_tags = main.find_elements(By.XPATH, ".//a[@href]")
    base_url = measure_url
    raw_links = [urljoin(base_url, a.get_attribute("href")) for a in a_tags]
    filtered = [u for u in raw_links if u.lower().endswith((".htm", ".pdf"))]

    # Prefer .htm if both .htm and .pdf exist for same base
    unique_docs = {}
    # for link in filtered:
    #     path = urlparse(link).path
    #     base = os.path.splitext(os.path.basename(path))[0]
    #     key = os.path.dirname(path) + "/" + base
    #     ext = os.path.splitext(path)[1].lower()
    #     if ext == ".htm":
    #         unique_docs[key] = link
    #     elif ext == ".pdf" and key not in unique_docs:
    #         unique_docs[key] = link

    results = []
    for doc_url in unique_docs.values():
        print(f"Processing: {doc_url}")
        if doc_url.lower().endswith(".htm"):
            driver.get(doc_url)
            time.sleep(3)
            html = driver.page_source
            text = clean_html_text(html)
            results.append({"url": doc_url, "text": text})
        elif doc_url.lower().endswith(".pdf"):
            # Remove old files first
            for f in os.listdir(download_dir):
                os.remove(os.path.join(download_dir, f))
            # Click the PDF link
            driver.get(measure_url)  # Reload base page to stay consistent
            time.sleep(3)
            link_el = driver.find_element(By.XPATH, f'//a[@href="{urlparse(doc_url).path}"]')
            link_el.click()
            time.sleep(5)  # Wait for download

            # Find the downloaded PDF file
            downloaded_pdf = next((os.path.join(download_dir, f)
                                   for f in os.listdir(download_dir)
                                   if f.lower().endswith(".pdf")), None)
            if downloaded_pdf:
                text = extract_pdf_text_from_file(downloaded_pdf)
                results.append({"url": doc_url, "text": text})
            else:
                results.append({"url": doc_url, "text": "[ERROR: PDF not downloaded]"})
    # Process starting page first
    driver.get(measure_url)
    time.sleep(0.5)
    html = driver.page_source
    text = table_html_to_numbered_list(html)
    links = extract_measure_links(html, measure_url)
    documents = extract_measure_documents_with_links(html, measure_url)
    results.append({"url": measure_url, "text": text, "links": links, "documents": documents})
    # Save results
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"✅ Saved to {output_filename}")

finally:
    driver.quit()
    shutil.rmtree(download_dir)


✅ Saved to HB_400_2025.json


In [5]:
%config PlainTextFormatter.max_width = 300 # Adjust the number as desired


In [6]:
print(results[0]['text'])
for row in results[0]['text']:
    print(row)
    print("-"*100)

['1. 6/26/2025 | H | Act 227, on 06/26/2025 (Gov. Msg. No. 1329).', '2. 6/26/2025 | S | Act 227, 06/26/2025 (Gov. Msg. No. 1329).', '3. 5/1/2025 | H | Transmitted to Governor.', '4. 5/2/2025 | S | Received notice of passage on Final Reading in House (Hse. Com. No. 821).', '5. 5/1/2025 | H | Received notice of Final Reading (Sen. Com. No. 888).', '6. 4/30/2025 | H | Passed Final Reading as amended in CD 1 with none voting aye with reservations; none voting no (0) and Representative(s) Cochran, Pierick excused (2).', '7. 4/30/2025 | S | Passed Final Reading, as amended (CD 1). Ayes, 25; Aye(s) with reservations: none . 0 No(es): none.  0 Excused: none.', '8. 4/25/2025 | S | 48 Hrs. Notice (as amended CD 1) 04-30-25', '9. 4/25/2025 | S | Reported from Conference Committee as amended CD 1 (Conf. Com. Rep. No. 157).', '10. 4/25/2025 | H | Forty-eight (48) hours notice Wednesday, 04-30-25.', '11. 4/25/2025 | H | Reported from Conference Committee (Conf Com. Rep. No. 157) as amended in (CD 1)

In [7]:
for row in results[0]['documents']:
    print(row)

{'name': 'HB400_SD2', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_SD2_.HTM'}
{'name': 'HB400_CD1', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_CD1_.HTM'}
{'name': 'HB400_HD1', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_HD1_.HTM'}
{'name': 'HB400_SD1', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_SD1_.HTM'}
{'name': 'HB400', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_.HTM'}
{'name': 'HB400_HSCR286_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/CommReports/HB400_HSCR286_.htm'}
{'name': 'HB400_HD1_HSCR1171_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/CommReports/HB400_HD1_HSCR1171_.htm'}
{'name': 'HB400_SD1_SSCR1253_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/CommReports/HB400_SD1_SSCR1253_.htm'}
{'name': 'HB400_SD2_SSCR1841_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/CommReports

In [8]:
import json
import re
from pydantic import BaseModel
from google import genai
class Document(BaseModel):
    date: str
    text: str
    documents: list[str]

# Updated Gemini query function
def query_gemini(prompt: str):
    client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents=prompt,
        config={"response_mime_type": "application/json", "response_schema": list[Document]}
    )
    return response.text, response.parsed

def generate_order_prompt_json(status_rows, document_names):
    """
    Generates a prompt asking Gemini 2.5 Pro to return a chronological JSON timeline of document creation.
    Each event should be an object with:
      - date: string (single date or date range)
      - text: string (description of the event)
      - documents: array of document names (can be empty)
    Include all documents. Place testimony documents under the relevant hearing event.
    """
    status_text = "\n".join(status_rows)
    documents_text = "\n".join(document_names)
    
    prompt = (
        "You are given legislative status updates of a bill and a list of document names.\n\n"
        "Some documents are testimonies related to hearings. Place testimony documents under the event where the hearing occurred.\n\n"
        "Status updates (chronological):\n"
        f"{status_text}\n\n"
        "Document names:\n"
        f"{documents_text}\n\n"
        "Return a chronological timeline as a JSON array of objects. Each object must have:\n"
        "{\n"
        '  "date": "event date or date range",\n'
        '  "text": "description of the event",\n'
        '  "documents": ["doc1", "doc2"]  # list of documents associated with this event, can be empty\n'
        "}\n\n"
        "Use all documents from the list. Do NOT include explanations, reasoning, or extra text. Just return valid JSON."
    )
    
    return prompt



documents = [document['name'] for document in results[0]['documents']]

# Generate the prompt
query = generate_order_prompt_json(results[0]['text'], documents)
print(query)

# Send prompt to Gemini
text, parsed = query_gemini(query)
print("******************\n\n")
print(text)
print("******************\n\n")
print(parsed)

timeline = parsed

doc_map = {doc['name']: doc for doc in results[0]['documents']}

documents_chronological = []
seen = set()
for event in timeline:
    for doc_name in event.documents:
        if doc_name in doc_map and doc_name not in seen:
            documents_chronological.append(doc_map[doc_name])
            seen.add(doc_name)

print(documents_chronological)



You are given legislative status updates of a bill and a list of document names.

Some documents are testimonies related to hearings. Place testimony documents under the event where the hearing occurred.

Status updates (chronological):
1. 6/26/2025 | H | Act 227, on 06/26/2025 (Gov. Msg. No. 1329).
2. 6/26/2025 | S | Act 227, 06/26/2025 (Gov. Msg. No. 1329).
3. 5/1/2025 | H | Transmitted to Governor.
4. 5/2/2025 | S | Received notice of passage on Final Reading in House (Hse. Com. No. 821).
5. 5/1/2025 | H | Received notice of Final Reading (Sen. Com. No. 888).
6. 4/30/2025 | H | Passed Final Reading as amended in CD 1 with none voting aye with reservations; none voting no (0) and Representative(s) Cochran, Pierick excused (2).
7. 4/30/2025 | S | Passed Final Reading, as amended (CD 1). Ayes, 25; Aye(s) with reservations: none . 0 No(es): none.  0 Excused: none.
8. 4/25/2025 | S | 48 Hrs. Notice (as amended CD 1) 04-30-25
9. 4/25/2025 | S | Reported from Conference Committee as amende

In [11]:
for document in documents_chronological:
    print(document)


{'name': 'HB400', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_.HTM'}
{'name': 'HB400_TESTIMONY_JHA_01-30-25_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/Testimony/HB400_TESTIMONY_JHA_01-30-25_.PDF'}
{'name': 'HB400_HSCR286_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/CommReports/HB400_HSCR286_.htm'}
{'name': 'HB400_TESTIMONY_FIN_03-05-25_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/Testimony/HB400_TESTIMONY_FIN_03-05-25_.PDF'}
{'name': 'HB400_HD1', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_HD1_.HTM'}
{'name': 'HB400_HD1_HSCR1171_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/CommReports/HB400_HD1_HSCR1171_.htm'}
{'name': 'HB400_HD1_TESTIMONY_JDC_03-19-25_', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/Testimony/HB400_HD1_TESTIMONY_JDC_03-19-25_.PDF'}
{'name': 'HB400_SD1', 'url': 'https://www.capitol.hawaii.gov/sessions/session2025/bills/HB400_SD

In [16]:
import os
import time
import fitz  # PyMuPDF
import shutil
import tempfile
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

def parse_web_document_selenium(url, output_dir="parsed_docs"):
    """
    Fetch a single HTML or PDF page using Selenium (undetected) to bypass Cloudflare,
    extract text, and save it as a .txt file.
    """
    os.makedirs(output_dir, exist_ok=True)
    if url.lower().endswith((".htm", ".html")):
        #skip
        return f"Skipping HTML file: {url}"
    
    # Derive a safe filename from URL
    path = urlparse(url).path
    filename_base = os.path.basename(path) or "document"
    txt_filename = os.path.join(output_dir, f"{filename_base}.txt")

    # Setup temp download directory for PDFs
    download_dir = tempfile.mkdtemp()

    options = uc.ChromeOptions()
    options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True
    })

    driver = uc.Chrome(options=options)

    def clean_html_text(html):
        soup = BeautifulSoup(html, "html.parser")
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()
        return soup.get_text(separator="\n", strip=True)

    def extract_pdf_text(file_path):
        try:
            doc = fitz.open(file_path)
            text = "\n".join(page.get_text() for page in doc)
            doc.close()
            return text
        except Exception as e:
            return f"[ERROR extracting PDF text: {e}]"

    try:
        driver.get(url)
        time.sleep(0.5)  # wait for Cloudflare / JS

        # if url.lower().endswith((".htm", ".html")):
        #     html = driver.page_source
        #     text = clean_html_text(html)

        if url.lower().endswith(".pdf"):
            # Navigate directly to the PDF URL
            driver.get(url)
            time.sleep(5)  # give time for PDF to download

            # Find the downloaded PDF
            downloaded_pdf = next((os.path.join(download_dir, f)
                                for f in os.listdir(download_dir)
                                if f.lower().endswith(".pdf")), None)
            if downloaded_pdf:
                text = extract_pdf_text(downloaded_pdf)
            else:
                return f"❌ PDF not downloaded: {url}"


        else:
            return f"Unsupported file type: {url}"

        # Save text
        with open(txt_filename, "w", encoding="utf-8") as f:
            f.write(text)

        return f"✅ Saved text: {txt_filename}"

    except Exception as e:
        return f"❌ Failed to parse {url}: {e}"

    finally:
        driver.quit()
        shutil.rmtree(download_dir)


for doc in documents_chronological:
    parse_web_document_selenium(doc['url'])


In [ ]:
import json
import os

# Example LLM query function (replace with your actual model)
def query_gemini(prompt: str):
    # Replace with your LLM call
    # return model.generate_content(prompt).text
    return f"[FAKE LLM RESPONSE FOR PROMPT: {prompt[:50]}...]"

# Property prompts (from your input)
PROPERTY_PROMPTS = { "overview": { 
    "prompt": "Using the provided legislative documents, statutes, and testimonies, write a clear summary describing the purpose, scope, and key components of the proposed measure or bill, including any pilot or permanent programs, reporting requirements, and sunset clauses. This should be around 3 sentences.", "description": "General overview and summary of the measure" }, 
    "appropriations": { "prompt": "Based on budgetary data and legislative appropriations, detail the funding allocated for the program or measure, including fiscal years, amounts, intended uses such as staffing, training, contracts, technology, etc... This should be around 3 sentences.", "description": "Funding allocation and appropriations details" }, 
    "assumptions_and_methodology": { "prompt": "Explain the assumptions, cost estimation methods, and data sources used to calculate the financial projections for this program or measure, referencing comparable programs or historical budgets where applicable. This should be around 3 sentences.", "description": "Cost estimation methodology and assumptions" }, 
    "agency_impact": { "prompt": "Describe the anticipated operational, administrative, and budgetary impact of the program or measure on the relevant government agency or department, including supervision, staffing, and resource allocation. This should be around 3 sentences.", "description": "Impact on government agencies and departments" }, 
    "economic_impact": { "prompt": "Summarize the expected economic effects of the program or measure, such as cost savings, potential reductions in related expenditures, benefits to the community, and any relevant performance or participation statistics. This should be around 3 sentences.", "description": "Economic effects and community benefits" }, 
    "policy_impact": { "prompt": "Analyze the policy implications of the measure, including how it modifies existing laws or programs, its role within broader legislative strategies, and its potential effects on state or local governance. This should be around 3 sentences.", "description": "Policy implications and legislative analysis" }, 
    "revenue_sources": { "prompt": "Identify and describe the funding sources that will support the program or measure, such as general funds, grants, fees, or other revenue streams, based on the provided fiscal documents. This should be around 3 sentences.", "description": "Funding sources and revenue streams" }, 
    "six_year_fiscal_implications": { "prompt": "Provide a multi-year fiscal outlook (e.g., six years) for the program or measure, projecting costs, staffing changes, recurring expenses, and assumptions about program expansion or permanence using available budget and workload data. This should be around 10 sentences.", "description": "Six-year fiscal projections and outlook" }, 
    "operating_revenue_impact": { "prompt": "Describe any anticipated impacts on operating revenues resulting from the program or measure, including increases, decreases, or changes in revenue streams. This should be around 3 sentences.", "description": "Operating revenue impacts" }, 
    "capital_expenditure_impact": { "prompt": "Outline any expected capital expenditures related to the program or measure, such as investments in facilities, equipment, or technology infrastructure, based on capital budgets or agency plans. This should be around 3 sentences.", "description": "Capital expenditure requirements" }, 
    "fiscal_implications_after_6_years": { "prompt": "Summarize the ongoing fiscal obligations after the initial multi-year period for the program or measure, including annual operating costs, expected number of program sites or units, and the sustainability of funding. This should be around 3 sentences.", "description": "Long-term fiscal obligations beyond six years" } 
    }

def generate_fiscal_note_for_context(context_text):
    """
    Generate a fiscal note for the given context using PROPERTY_PROMPTS.
    Returns a dictionary where keys are property names and values are LLM outputs.
    """
    fiscal_note = {}
    for key, prop in PROPERTY_PROMPTS.items():
        prompt = f"{prop['prompt']}\n\nContext:\n{context_text}"
        response = query_gemini(prompt)
        fiscal_note[key] = response
    return fiscal_note

def generate_fiscal_notes_chronologically(documents, output_dir="fiscal_notes"):
    """
    Generate fiscal notes sequentially for a list of chronologically ordered documents.
    Each document adds to the cumulative context.
    
    documents: list of dicts with {"name": ..., "text": ...}
    """
    os.makedirs(output_dir, exist_ok=True)
    cumulative_context = ""
    
    for i, doc in enumerate(documents, start=1):
        print(f"Processing document {i}/{len(documents)}: {doc['name']}")
        
        # Append the new document to the cumulative context
        cumulative_context += f"\n\n=== Document: {doc['name']} ===\n{doc['text']}"
        
        # Generate fiscal note for the current cumulative context
        fiscal_note = generate_fiscal_note_for_context(cumulative_context)
        
        # Save to a JSON file (filename = new document name)
        out_path = os.path.join(output_dir, f"{doc['name']}.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(fiscal_note, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Fiscal note saved: {out_path}")

